# Model Complexity Analysis

JeongWonNet vs CMUNeXt 복잡도 비교 분석

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

from thop import profile
from models import JeongWonNet, CMUNeXt

INPUT_SIZE = (1, 3, 256, 256)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 1. 전체 비교 요약

In [ ]:
def get_model_stats(model, name, input_size):
    """모델 통계 계산"""
    dummy = torch.randn(input_size).to(device)
    model = model.to(device)
    
    # Parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    # FLOPs
    flops, _ = profile(model, inputs=(dummy,), verbose=False)
    
    # Model size (FP32)
    size_mb = total_params * 4 / (1024 ** 2)
    
    return {
        'Model': name,
        'Total Params': total_params,
        'Trainable Params': trainable_params,
        'FLOPs': flops,
        'Size (MB)': size_mb
    }

# 모델 생성
model1 = JeongWonNet(num_classes=1)
model2 = CMUNeXt(num_classes=1)

stats1 = get_model_stats(model1, 'JeongWonNet', INPUT_SIZE)
stats2 = get_model_stats(model2, 'CMUNeXt', INPUT_SIZE)

# 비교 테이블
df_summary = pd.DataFrame([stats1, stats2])
df_summary['Params Ratio'] = df_summary['Total Params'] / df_summary['Total Params'].min()
df_summary['FLOPs Ratio'] = df_summary['FLOPs'] / df_summary['FLOPs'].min()

print("="*60)
print("Overall Comparison")
print("="*60)
print(f"\n{'Metric':<20} {'JeongWonNet':>15} {'CMUNeXt':>15} {'Ratio':>10}")
print("-"*60)
print(f"{'Parameters':<20} {stats1['Total Params']:>15,} {stats2['Total Params']:>15,} {stats2['Total Params']/stats1['Total Params']:>10.1f}x")
print(f"{'FLOPs':<20} {stats1['FLOPs']:>15,.0f} {stats2['FLOPs']:>15,.0f} {stats2['FLOPs']/stats1['FLOPs']:>10.1f}x")
print(f"{'Size (MB)':<20} {stats1['Size (MB)']:>15.3f} {stats2['Size (MB)']:>15.3f} {stats2['Size (MB)']/stats1['Size (MB)']:>10.1f}x")

## 2. 레이어별 파라미터 분석

In [ ]:
def analyze_layers(model, name):
    """레이어별 파라미터 분석"""
    print(f"\n{'='*60}")
    print(f"{name} - Layer Analysis")
    print(f"{'='*60}")
    
    layer_stats = []
    total = 0
    
    for name_layer, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.Linear, nn.BatchNorm2d, nn.GroupNorm, nn.LayerNorm)):
            params = sum(p.numel() for p in module.parameters())
            if params > 0:
                layer_stats.append({
                    'Layer': name_layer,
                    'Type': module.__class__.__name__,
                    'Params': params
                })
                total += params
    
    df = pd.DataFrame(layer_stats)
    df['Percentage'] = df['Params'] / total * 100
    
    # 타입별 집계
    type_summary = df.groupby('Type')['Params'].sum().sort_values(ascending=False)
    
    print(f"\n[By Layer Type]")
    print(f"{'Type':<20} {'Params':>12} {'Percentage':>10}")
    print("-"*45)
    for t, p in type_summary.items():
        print(f"{t:<20} {p:>12,} {p/total*100:>9.1f}%")
    print("-"*45)
    print(f"{'Total':<20} {total:>12,}")
    
    return df, type_summary

df1, type1 = analyze_layers(model1, 'JeongWonNet')
df2, type2 = analyze_layers(model2, 'CMUNeXt')

## 3. Conv2d 레이어 상세 비교

In [ ]:
def analyze_conv_layers(model, name):
    """Conv2d 레이어 상세 분석"""
    print(f"\n{'='*60}")
    print(f"{name} - Conv2d Analysis")
    print(f"{'='*60}")
    
    conv_stats = []
    
    for name_layer, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            params = sum(p.numel() for p in module.parameters())
            is_depthwise = module.groups == module.in_channels and module.groups > 1
            is_pointwise = module.kernel_size == (1, 1)
            
            conv_type = 'Depthwise' if is_depthwise else ('Pointwise' if is_pointwise else 'Standard')
            
            conv_stats.append({
                'Layer': name_layer.split('.')[-1] if '.' in name_layer else name_layer,
                'Type': conv_type,
                'In': module.in_channels,
                'Out': module.out_channels,
                'Kernel': module.kernel_size[0],
                'Groups': module.groups,
                'Params': params
            })
    
    df = pd.DataFrame(conv_stats)
    
    # 타입별 통계
    print(f"\n[Conv Type Distribution]")
    type_dist = df.groupby('Type').agg({'Params': ['count', 'sum']})
    type_dist.columns = ['Count', 'Total Params']
    type_dist['Avg Params'] = type_dist['Total Params'] / type_dist['Count']
    print(type_dist.to_string())
    
    # 커널 크기별 통계
    print(f"\n[Kernel Size Distribution]")
    kernel_dist = df.groupby('Kernel').agg({'Params': ['count', 'sum']})
    kernel_dist.columns = ['Count', 'Total Params']
    print(kernel_dist.to_string())
    
    return df

conv1 = analyze_conv_layers(model1, 'JeongWonNet')
conv2 = analyze_conv_layers(model2, 'CMUNeXt')

## 4. 채널 수 비교

In [ ]:
def get_channel_config(model, name):
    """모델의 채널 구성 추출"""
    channels = []
    
    for n, m in model.named_modules():
        if isinstance(m, nn.Conv2d):
            if m.in_channels not in channels:
                channels.append(m.in_channels)
            if m.out_channels not in channels:
                channels.append(m.out_channels)
    
    channels = sorted(set(channels))
    return channels

ch1 = get_channel_config(model1, 'JeongWonNet')
ch2 = get_channel_config(model2, 'CMUNeXt')

print("\n" + "="*60)
print("Channel Configuration")
print("="*60)
print(f"\nJeongWonNet channels: {ch1}")
print(f"Max channel: {max(ch1)}")
print(f"\nCMUNeXt channels: {ch2}")
print(f"Max channel: {max(ch2)}")
print(f"\nMax channel ratio: {max(ch2)/max(ch1):.1f}x")

## 5. Encoder/Decoder 구조 비교

In [ ]:
def analyze_encoder_decoder(model, name):
    """Encoder/Decoder 파라미터 분포"""
    encoder_params = 0
    decoder_params = 0
    other_params = 0
    
    for n, p in model.named_parameters():
        params = p.numel()
        if 'encoder' in n.lower():
            encoder_params += params
        elif 'decoder' in n.lower():
            decoder_params += params
        else:
            other_params += params
    
    total = encoder_params + decoder_params + other_params
    
    print(f"\n{name}:")
    print(f"  Encoder: {encoder_params:>10,} ({encoder_params/total*100:>5.1f}%)")
    print(f"  Decoder: {decoder_params:>10,} ({decoder_params/total*100:>5.1f}%)")
    print(f"  Other:   {other_params:>10,} ({other_params/total*100:>5.1f}%)")
    print(f"  Total:   {total:>10,}")
    
    return {'encoder': encoder_params, 'decoder': decoder_params, 'other': other_params}

print("="*60)
print("Encoder/Decoder Distribution")
print("="*60)

ed1 = analyze_encoder_decoder(model1, 'JeongWonNet')
ed2 = analyze_encoder_decoder(model2, 'CMUNeXt')

## 6. 복잡도 효율성 비교

In [ ]:
# 실제 측정 결과 (measure_efficiency.ipynb에서)
measured = {
    'JeongWonNet': {'params': 9650, 'flops': 11.55e6, 'gpu_ms': 0.700, 'cpu_ms': 2.033},
    'CMUNeXt': {'params': 144980, 'flops': 776.55e6, 'gpu_ms': 1.454, 'cpu_ms': 8.025}
}

print("="*60)
print("Efficiency Comparison")
print("="*60)

j = measured['JeongWonNet']
c = measured['CMUNeXt']

print(f"\n{'Metric':<25} {'JeongWonNet':>12} {'CMUNeXt':>12} {'Reduction':>12}")
print("-"*65)
print(f"{'Parameters':<25} {j['params']:>12,} {c['params']:>12,} {c['params']/j['params']:>11.1f}x")
print(f"{'FLOPs':<25} {j['flops']/1e6:>11.2f}M {c['flops']/1e6:>11.2f}M {c['flops']/j['flops']:>11.1f}x")
print(f"{'GPU Latency (ms)':<25} {j['gpu_ms']:>12.3f} {c['gpu_ms']:>12.3f} {c['gpu_ms']/j['gpu_ms']:>11.1f}x")
print(f"{'CPU Latency (ms)':<25} {j['cpu_ms']:>12.3f} {c['cpu_ms']:>12.3f} {c['cpu_ms']/j['cpu_ms']:>11.1f}x")
print(f"{'FLOPs/Param':<25} {j['flops']/j['params']:>12.1f} {c['flops']/c['params']:>12.1f} {(c['flops']/c['params'])/(j['flops']/j['params']):>11.1f}x")
print(f"{'Throughput (GPU FPS)':<25} {1000/j['gpu_ms']:>12.1f} {1000/c['gpu_ms']:>12.1f} {(1000/j['gpu_ms'])/(1000/c['gpu_ms']):>11.1f}x")
print(f"{'Throughput (CPU FPS)':<25} {1000/j['cpu_ms']:>12.1f} {1000/c['cpu_ms']:>12.1f} {(1000/j['cpu_ms'])/(1000/c['cpu_ms']):>11.1f}x")

## 7. 시각화

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

models = ['JeongWonNet', 'CMUNeXt']
colors = ['#2ecc71', '#3498db']

# 1. Parameters
ax1 = axes[0, 0]
params = [j['params'], c['params']]
bars = ax1.bar(models, params, color=colors)
ax1.set_ylabel('Parameters')
ax1.set_title('Parameters Comparison')
for bar, val in zip(bars, params):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{val:,}', 
             ha='center', va='bottom', fontsize=10)

# 2. FLOPs
ax2 = axes[0, 1]
flops = [j['flops']/1e6, c['flops']/1e6]
bars = ax2.bar(models, flops, color=colors)
ax2.set_ylabel('FLOPs (M)')
ax2.set_title('FLOPs Comparison')
for bar, val in zip(bars, flops):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{val:.1f}M', 
             ha='center', va='bottom', fontsize=10)

# 3. Latency
ax3 = axes[1, 0]
x = np.arange(2)
width = 0.35
gpu = [j['gpu_ms'], c['gpu_ms']]
cpu = [j['cpu_ms'], c['cpu_ms']]
bars1 = ax3.bar(x - width/2, gpu, width, label='GPU', color='#e74c3c')
bars2 = ax3.bar(x + width/2, cpu, width, label='CPU', color='#9b59b6')
ax3.set_ylabel('Latency (ms)')
ax3.set_title('Inference Latency')
ax3.set_xticks(x)
ax3.set_xticklabels(models)
ax3.legend()

# 4. Reduction Ratio
ax4 = axes[1, 1]
metrics = ['Params', 'FLOPs', 'GPU', 'CPU']
ratios = [c['params']/j['params'], c['flops']/j['flops'], 
          c['gpu_ms']/j['gpu_ms'], c['cpu_ms']/j['cpu_ms']]
bars = ax4.barh(metrics, ratios, color='#f39c12')
ax4.set_xlabel('CMUNeXt / JeongWonNet Ratio')
ax4.set_title('Complexity Ratio (CMUNeXt vs JeongWonNet)')
ax4.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
for bar, val in zip(bars, ratios):
    ax4.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f'{val:.1f}x', 
             ha='left', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('complexity_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nSaved: complexity_analysis.png")

## 8. 결론

In [ ]:
print("="*60)
print("Summary")
print("="*60)
print(f"""
JeongWonNet vs CMUNeXt 복잡도 비교:

1. Parameters: JeongWonNet이 {c['params']/j['params']:.1f}x 더 작음
   - JeongWonNet: {j['params']:,} params
   - CMUNeXt: {c['params']:,} params

2. FLOPs: JeongWonNet이 {c['flops']/j['flops']:.1f}x 더 적은 연산
   - JeongWonNet: {j['flops']/1e6:.2f}M FLOPs
   - CMUNeXt: {c['flops']/1e6:.2f}M FLOPs

3. GPU Latency: JeongWonNet이 {c['gpu_ms']/j['gpu_ms']:.1f}x 더 빠름
   - JeongWonNet: {j['gpu_ms']:.3f} ms
   - CMUNeXt: {c['gpu_ms']:.3f} ms

4. CPU Latency: JeongWonNet이 {c['cpu_ms']/j['cpu_ms']:.1f}x 더 빠름
   - JeongWonNet: {j['cpu_ms']:.3f} ms
   - CMUNeXt: {c['cpu_ms']:.3f} ms

핵심 차이점:
- JeongWonNet: Depthwise Separable Conv + 작은 채널 수
- CMUNeXt: 더 큰 채널 수와 복잡한 구조
""")